# from flask import Flask, render_template, request
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

def scrape_amazon(query):
    headers = {"User-Agent": "Mozilla/5.0"}
    url = f"https://www.amazon.com/s?k={query}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = []
    for item in soup.select('.s-main-slot .s-result-item'):
        title = item.select_one('h2').get_text() if item.select_one('h2') else 'No title'
        link = item.select_one('a')['href'] if item.select_one('a') else 'No link'
        products.append({'title': title, 'link': 'https://www.amazon.com' + link})
    return products

def scrape_youtube(query):
    headers = {"User-Agent": "Mozilla/5.0"}
    url = f"https://www.youtube.com/results?search_query={query}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    videos = []
    for item in soup.select('a[href^="/watch"]'):
        title = item.get('title')
        link = 'https://www.youtube.com' + item.get('href')
        videos.append({'title': title, 'link': link})
    return videos

@app.route('/', methods=['GET', 'POST'])
def index():
    amazon_results = []
    youtube_results = []
    if request.method == 'POST':
        query = request.form.get('query')
        amazon_results = scrape_amazon(query)
        youtube_results = scrape_youtube(query)
    return render_template('index.html', amazon_results=amazon_results, youtube_results=youtube_results)

if __name__ == '__main__':
    app.run(debug=True)


In [4]:
                    ''''q2'''

"'q2"

# from flask import Flask, render_template, request
import requests

app = Flask(__name__)

API_KEY = 'your_openweathermap_api_key'  # Replace with your actual API key
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'

def get_weather(city):
    params = {
        'q': city,
        'appid': API_KEY,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    if data.get('cod') != 200:
        return None
    weather = {
        'city': data['name'],
        'description': data['weather'][0]['description'],
        'temperature': data['main']['temp'],
        'humidity': data['main']['humidity'],
        'pressure': data['main']['pressure']
    }
    return weather

@app.route('/', methods=['GET', 'POST'])
def index():
    weather = None
    if request.method == 'POST':
        city = request.form.get('city')
        weather = get_weather(city)
    return render_template('index.html', weather=weather)

if __name__ == '__main__':
    app.run(debug=True)






In [10]:
                    '''q3'''

'q3'

# from flask import Flask, redirect, url_for, session, request, render_template
from authlib.integrations.flask_client import OAuth
import os

app = Flask(__name__)
app.secret_key = os.environ.get('SECRET_KEY', 'your_secret_key')
app.config['SESSION_COOKIE_NAME'] = 'my_flask_session'

# OAuth configuration
oauth = OAuth(app)

# Google OAuth2 setup
google = oauth.register(
    name='google',
    client_id=os.environ.get('GOOGLE_CLIENT_ID'),
    client_secret=os.environ.get('GOOGLE_CLIENT_SECRET'),
    authorize_url='https://accounts.google.com/o/oauth2/auth',
    authorize_params=None,
    access_token_url='https://accounts.google.com/o/oauth2/token',
    access_token_params=None,
    refresh_token_url=None,
    redirect_uri='http://localhost:5000/auth/google/callback',
    client_kwargs={'scope': 'openid profile email'},
)

# Facebook OAuth2 setup
facebook = oauth.register(
    name='facebook',
    client_id=os.environ.get('FACEBOOK_CLIENT_ID'),
    client_secret=os.environ.get('FACEBOOK_CLIENT_SECRET'),
    authorize_url='https://www.facebook.com/dialog/oauth',
    authorize_params=None,
    access_token_url='https://graph.facebook.com/oauth/access_token',
    access_token_params=None,
    refresh_token_url=None,
    redirect_uri='http://localhost:5000/auth/facebook/callback',
    client_kwargs={'scope': 'email'},
)

@app.route('/')
def index():
    user = session.get('user')
    return render_template('index.html', user=user)

@app.route('/login/google')
def login_google():
    redirect_uri = url_for('auth_google', _external=True)
    return google.authorize_redirect(redirect_uri)

@app.route('/auth/google/callback')
def auth_google():
    token = google.authorize_access_token()
    user_info = google.parse_id_token(token)
    session['user'] = user_info
    return redirect('/')

@app.route('/login/facebook')
def login_facebook():
    redirect_uri = url_for('auth_facebook', _external=True)
    return facebook.authorize_redirect(redirect_uri)

@app.route('/auth/facebook/callback')
def auth_facebook():
    token = facebook.authorize_access_token()
    user_info = facebook.parse_id_token(token)
    session['user'] = user_info
    return redirect('/')

@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect('/')

if __name__ == '__main__':
    app.run(debug=True)



In [13]:
                    '''q4'''

'q4'

# from flask import Flask, render_template, request

app = Flask(__name__)

# Sample content data
CONTENT = [
    {'id': 1, 'title': 'Introduction to Python', 'category': 'Programming'},
    {'id': 2, 'title': 'Advanced Python Techniques', 'category': 'Programming'},
    {'id': 3, 'title': 'Understanding Machine Learning', 'category': 'Data Science'},
    {'id': 4, 'title': 'Deep Learning with TensorFlow', 'category': 'Data Science'},
    {'id': 5, 'title': 'HTML & CSS Basics', 'category': 'Web Development'},
    {'id': 6, 'title': 'JavaScript Essentials', 'category': 'Web Development'},
]

def get_recommendations(preferences):
    # Simple recommendation logic based on category preferences
    recommended = [content for content in CONTENT if content['category'] in preferences]
    return recommended

@app.route('/', methods=['GET', 'POST'])
def index():
    recommendations = []
    if request.method == 'POST':
        preferences = request.form.getlist('preferences')
        recommendations = get_recommendations(preferences)
    return render_template('index.html', recommendations=recommendations)

if __name__ == '__main__':
    app.run(debug=True)
